In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
import open_clip
open_clip.list_pretrained()
import torch
from PIL import Image

In [2]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [3]:
# model, train_preprocess, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='datacomp_xl_s13b_b90k')
# model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
# tokenizer = open_clip.get_tokenizer('ViT-B-16')

model, train_preprocess, preprocess = open_clip.create_model_and_transforms('MobileCLIP-S1', pretrained='datacompdr')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('MobileCLIP-S1')

In [3]:
# model, train_preprocess, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='datacomp_xl_s13b_b90k')
# model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
# tokenizer = open_clip.get_tokenizer('ViT-B-16')

model, train_preprocess, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='datacomp_xl_s13b_b90k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('ViT-B-16')

In [4]:
train_preprocess

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    <function _convert_to_rgb at 0x74dd759b92d0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [5]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_to_rgb at 0x74dd759b92d0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [4]:
model

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): FastVit(
      (stem): Sequential(
        (0): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList(
            (0): ConvNormAct(
              (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (bn): BatchNormAct2d(
                64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
                (drop): Identity()
                (act): Identity()
              )
            )
          )
          (conv_scale): ConvNormAct(
            (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn): BatchNormAct2d(
              64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
          )
          (act): GELU(approximate='none')
        )
        (1): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList

In [7]:
# model.visual.proj.shape

AttributeError: 'TimmModel' object has no attribute 'proj'

In [8]:
# model.visual.pool_type

AttributeError: 'TimmModel' object has no attribute 'pool_type'

In [7]:
# print(model.visual.attn_pool)

None


In [8]:
# model.visual.final_ln_after_pool

False

In [9]:
model.visual.output_tokens = True

In [10]:
model.text_pool_type

AttributeError: 'CustomTextCLIP' object has no attribute 'text_pool_type'

In [11]:
model.text_projection.shape

AttributeError: 'CustomTextCLIP' object has no attribute 'text_projection'

In [5]:
image = preprocess(Image.open("/home/alex/data/test_cap.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a young girl", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

stem:torch.Size([1, 64, 64, 64])
stage0:torch.Size([1, 64, 64, 64])
stage1:torch.Size([1, 128, 32, 32])
stage2:torch.Size([1, 256, 16, 16])
stage3:torch.Size([1, 512, 8, 8])
final_conv:torch.Size([1, 1024, 8, 8])
head:torch.Size([1, 512])
text0:torch.Size([77, 4, 512])
text1:torch.Size([77, 4, 512])
text2:torch.Size([77, 4, 512])
text3:torch.Size([77, 4, 512])
text4:torch.Size([77, 4, 512])
text5:torch.Size([77, 4, 512])
text6:torch.Size([77, 4, 512])
text7:torch.Size([77, 4, 512])
text8:torch.Size([77, 4, 512])
text9:torch.Size([77, 4, 512])
text10:torch.Size([77, 4, 512])
text11:torch.Size([77, 4, 512])
Label probs: tensor([[2.8675e-05, 1.9033e-03, 9.9389e-01, 4.1754e-03]])


ViT-B-16:每张图片分为196+1个patch（$(224/16)^2+1$），1个patch为cls token，image_features是取的这个cls token，其余的可通过token_features返回。

In [6]:
model.text.pool_type

'argmax'

In [7]:
from open_clip.model import Custom3DConvModule

In [8]:
model.visual.trunk.stem = torch.nn.Sequential(
    Custom3DConvModule(in_channels=3),
    model.visual.trunk.stem[0],
    Custom3DConvModule(in_channels=64),
    model.visual.trunk.stem[1],
    Custom3DConvModule(in_channels=64),
    model.visual.trunk.stem[2],
    Custom3DConvModule(in_channels=64),
)


In [10]:
model.visual.trunk.stages = torch.nn.Sequential(
    model.visual.trunk.stages[0],
    Custom3DConvModule(in_channels=64),
    model.visual.trunk.stages[1],
    Custom3DConvModule(in_channels=128),
    model.visual.trunk.stages[2],
    Custom3DConvModule(in_channels=256),
    model.visual.trunk.stages[3],
    Custom3DConvModule(in_channels=512),
)

In [1]:
import torch
import numpy as np
def get_sinusoid_encoding_table(n_position, d_hid):
    """Sinusoid position encoding table"""

    # TODO: make it with torch instead of numpy
    def get_position_angle_vec(position):
        return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

    return torch.FloatTensor(sinusoid_table).unsqueeze(0)

In [2]:
position_embeddings = get_sinusoid_encoding_table(n_position=4,d_hid=512)

In [4]:
position_embeddings.shape

torch.Size([1, 4, 512])

In [5]:
x = torch.randn(128,4,8*8,512)

In [7]:
x_emb = position_embeddings.type_as(x).to(x.device).clone().detach()

In [9]:
x_emb.shape

torch.Size([1, 4, 512])

In [13]:
y = (x.transpose(1,2) + x_emb).transpose(1,2)

In [14]:
y.shape

torch.Size([128, 4, 64, 512])

In [9]:
model

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): FastVit(
      (stem): Sequential(
        (0): Custom3DConvModule(
          (conv3d_1): Conv3d(3, 12, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
          (bn_act3d_1): BatchNormAct3d(
            12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv3d_2): Conv3d(12, 3, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
          (bn_act3d_2): BatchNormAct3d(
            3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
        )
        (1): MobileOneBlock(
          (se): Identity()
          (conv_kxk): ModuleList(
            (0): ConvNormAct(
              (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (bn): BatchNormAct2d(
                64, eps=

In [11]:
# token_features.shape

torch.Size([1, 196, 768])

In [15]:
image_features.shape

torch.Size([1, 512])

In [16]:
text_features.shape

torch.Size([4, 512])